In [53]:
#识别手写数字1-9实验
#实验概述：单层处理单个数字，共10层，每层采用逻辑回归的基本逻辑，并采用向量化的方式提高计算效率
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat     #读取 MATLAB.mat文件的函数

In [54]:
#读取训练数据
data=loadmat('ex3data1.mat')  
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [55]:
#X为像素点20*20的400维向量，共5000个
data['X'].shape,data['y'].shape

((5000, 400), (5000, 1))

In [124]:
#part1：向量化逻辑回归的部分
#定义sigmoid函数
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [125]:
#定义代价函数
#处理对象：神经网络单层参数，训练数据：全部 X与 y
def cost(theta,X,y,learningRate):
    theta = np.matrix(theta)
    X=np.matrix(X)
    y=np.matrix(y)
    first=np.multiply(-y,np.log(sigmoid(X*theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate/(2*len(X)))*np.sum(np.power(theta[:,1:],2))    #正则化
    return np.sum(first-second)/len(X)+reg      

In [126]:
#定义梯度
#为确保实验效率，采用向量化代替for循环
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    error = sigmoid(X * theta.T) - y    #计算参数误差

    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    grad[:,1:] = grad[:,1:] + ((learningRate / len(X)) * theta[:,1:])   #正则化处理参数下降梯度
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)    #对b进行去正则化
    
    return np.array(grad).ravel()      #返回向量化的所有参数的下降梯度

In [127]:
#part2：构建分类器
#策略1：一对多分类，创建10个二分类器，获得最佳参数矩阵

from scipy.optimize import minimize     #SciPy库中的优化函数，寻找最小值

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]          #定义样本数量5000
    params = X.shape[1]        #定义特征数量400
    all_theta = np.zeros((num_labels, params + 1))     #定义【10*401】参数矩阵
    
    #数据预处理，给X增加偏置项全0列
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    #核心循环训练过程（1-11）
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)        # 初始化当前分类器的参数
        y_i = np.array([1 if label[0] == i else 0 for label in y])    # 注意y是二维数组
        y_i = np.reshape(y_i, (rows, 1))   #重塑数据结构，将一维数组转换为【5000*1】二维数组，与X相匹配
        
        #优化函数，寻找cost最小的参数值
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x      #把当前数字分类器训练好的参数保存到结果矩阵中
   
    return all_theta

In [128]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [129]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-2.09140667e+00,  0.00000000e+00,  0.00000000e+00, ...,
         6.53907122e-04, -7.89033894e-10,  0.00000000e+00],
       [-3.10409152e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.80769082e-03, -3.20655012e-04,  0.00000000e+00],
       [-4.58029353e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.23645868e-05, -2.11542154e-07,  0.00000000e+00],
       ...,
       [-7.55676397e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -6.12852070e-05,  4.15003668e-06,  0.00000000e+00],
       [-4.34302285e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -5.27745995e-04,  3.85567157e-05,  0.00000000e+00],
       [-4.85810729e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.35489186e-04,  8.82812221e-06,  0.00000000e+00]])

In [121]:
def predict_all(X, all_theta):
    #数据处理，和之前保持一致
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    #计算X在参数模型中等于不同结果的对应概率，取最高为预测值
    h = sigmoid(X * all_theta.T)
    h_argmax = np.argmax(h, axis=1)
    h_argmax = h_argmax + 1    #[0-9]——>[1-10]
    
    return h_argmax

In [130]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 93.58%
